In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

from datetime import datetime

In [2]:
data = pd.read_excel('./데이터/Data_final_이전.xlsx')
data

,학교급,지역,교육청,설립유형,사고자구분,사고자성별,사고자학년,사고발생일,사고발생요일,사고발생시각,사고시간,사고장소,사고부위,사고형태,사고당시활동,사고매개물,사고연도,사고월,사고일,나이
0,중학교,경기,안양과천교육지원청,공립,일반학생,남,1학년,2019-01-02,수,09:00,수업시간,부속시설,다리,낙상-미끄러짐,기타운동,건물(문/창문/바닥/벽 등),2019,1,2,중학생
1,초등학교,경기,안양과천교육지원청,공립,일반학생,남,2학년,2019-01-02,수,08:50,등하교,통로,치아(구강),낙상-넘어짐,보행/주행,건물(문/창문/바닥/벽 등),2019,1,2,초등학생 저학년
2,중학교,경기,부천교육지원청,공립,일반학생,여,1학년,2019-01-02,수,10:20,체육수업,부속시설,손,기타,구기운동,기타,2019,1,2,중학생
3,초등학교,광주,서부교육지원청,공립,일반학생,여,2학년,2019-01-02,수,10:35,휴식시간 및 청소시간,운동장,머리(두부),낙상-넘어짐,장난/놀이,자연(사람/동물/식물 등),2019,1,2,초등학생 저학년
4,중학교,충북,청주교육지원청,공립,일반학생,남,2학년,2019-01-02,수,09:10,체육수업,부속시설,발,낙상-미끄러짐,보행/주행,건물(문/창문/바닥/벽 등),2019,1,2,중학생
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
611765,중학교,제주,서귀포시교육지원청,공립,일반학생,여,2학년,2023-12-11,월,14:10,쉬는시간,교실,팔,물리적힘 노출,장난/놀이,NaN,2023,12,11,중학생
611766,중학교,제주,제주시교육지원청,공립,일반학생,남,1학년,2023-12-08,금,13:20,쉬는시간,통로,다리,사람과의 충돌,식사/수면/휴식,NaN,2023,12,8,중학생
611767,중학교,제주,제주시교육지원청,공립,일반학생,여,3학년,2023-12-04,월,09:20,특별활동,운동장,손,물리적힘 노출,구기운동,NaN,2023,12,4,중학생
611768,중학교,제주,제주시교육지원청,공립,일반학생,남,1학년,2023-12-13,수,12:25,식사시간,부속시설,발,낙상,장난/놀이,NaN,2023,12,13,중학생


In [3]:
data_original = data.copy()

In [4]:
# data = data_original
# data

In [5]:
# '사고발생일' 컬럼 제거
if '사고발생일' in data.columns:
    data.drop(columns=['사고발생일'], inplace=True)

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 611770 entries, 0 to 611769
Data columns (total 19 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   학교급     611770 non-null  object
 1   지역      611770 non-null  object
 2   교육청     611770 non-null  object
 3   설립유형    611770 non-null  object
 4   사고자구분   611770 non-null  object
 5   사고자성별   611770 non-null  object
 6   사고자학년   610388 non-null  object
 7   사고발생요일  611770 non-null  object
 8   사고발생시각  611770 non-null  object
 9   사고시간    611770 non-null  object
 10  사고장소    611770 non-null  object
 11  사고부위    611770 non-null  object
 12  사고형태    611770 non-null  object
 13  사고당시활동  611770 non-null  object
 14  사고매개물   418596 non-null  object
 15  사고연도    611770 non-null  int64 
 16  사고월     611770 non-null  int64 
 17  사고일     611770 non-null  int64 
 18  나이      611770 non-null  object
dtypes: int64(3), object(16)
memory usage: 88.7+ MB


In [7]:
# 라벨 인코더 생성 및 null 값을 임시 대체하여 적용
label_encoders = {}
for column in data.select_dtypes(include=['object']).columns:
    label_encoders[column] = LabelEncoder()
    if column == '사고매개물':
        # "사고매개물" 컬럼의 null 값을 제외하고 라벨 인코딩
        non_null_mask = data[column].notnull()
        data.loc[non_null_mask, column] = label_encoders[column].fit_transform(data.loc[non_null_mask, column].astype(str))
    else:
        # 다른 컬럼은 null 값을 포함하여 라벨 인코딩
        data[column] = data[column].astype(str).fillna('NULL_PLACEHOLDER')
        data[column] = label_encoders[column].fit_transform(data[column])

In [8]:
# "사고매개물" 컬럼에 null 값이 있는 데이터와 없는 데이터 분리
data_null = data[data['사고매개물'].isnull()]
data_not_null = data[~data['사고매개물'].isnull()]

print(data_null.shape[0])
print(data_not_null.shape[0])

193174
418596


In [9]:
print(data)
print(data_null)
print(data_not_null)

        학교급  지역  교육청  설립유형  사고자구분  사고자성별  사고자학년  사고발생요일  사고발생시각  사고시간  사고장소  \
0         3   1   96     0      7      0      0       2     223     4     2   
1         4   1   96     0      7      0      1       2     213     2     4   
2         3   1   69     0      7      1      0       2     309     8     2   
3         4   4   76     0      7      1      1       2     325    12     3   
4         3  16  154     0      7      0      1       2     233     8     2   
...     ...  ..  ...   ...    ...    ...    ...     ...     ...   ...   ...   
611765    3  14   75     0      7      1      1       3     578     5     0   
611766    3  14  139     0      7      0      0       0     519     5     4   
611767    3  14  139     0      7      1      2       3     243    10     3   
611768    3  14  139     0      7      0      0       2     454     6     2   
611769    3  14  139     0      7      0      0       1     692     9     3   

        사고부위  사고형태  사고당시활동 사고매개물  사고연도  사고월  사고일  나

In [10]:
# null 값이 있는지 확인
if not data_null.empty:
    # 특징과 타겟 분리 (타겟: 사고매개물)
    X = data_not_null.drop(columns=['사고매개물'])
    y = data_not_null['사고매개물'].astype(int)

    # 훈련/테스트 데이터 분리
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # 모델 학습 (랜덤 포레스트)
    model = RandomForestClassifier(random_state=42)
    model.fit(X_train, y_train)

    # 모델 평가
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f'Model Accuracy: {accuracy}')

    # null 값 예측
    X_null = data_null.drop(columns=['사고매개물'])
    data_null['사고매개물'] = model.predict(X_null)

    # 예측된 값으로 원래 데이터 채우기
    data.loc[data['사고매개물'].isnull(), '사고매개물'] = data_null['사고매개물']
else:
    print("No null values found in '사고매개물' column.")
    
data

Model Accuracy: 0.5888795986622074


C:\Users\USER\AppData\Local\Temp\ipykernel_3460\1019422097.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_null['사고매개물'] = model.predict(X_null)


,학교급,지역,교육청,설립유형,사고자구분,사고자성별,사고자학년,사고발생요일,사고발생시각,사고시간,사고장소,사고부위,사고형태,사고당시활동,사고매개물,사고연도,사고월,사고일,나이
0,3,1,96,0,7,0,0,2,223,4,2,1,4,3,1,2019,1,2,5
1,4,1,96,0,7,0,1,2,213,2,4,6,2,4,1,2019,1,2,7
2,3,1,69,0,7,1,0,2,309,8,2,5,0,1,3,2019,1,2,5
3,4,4,76,0,7,1,1,2,325,12,3,2,2,7,8,2019,1,2,7
4,3,16,154,0,7,0,1,2,233,8,2,3,4,4,1,2019,1,2,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
611765,3,14,75,0,7,1,1,3,578,5,0,7,5,7,0,2023,12,11,5
611766,3,14,139,0,7,0,0,0,519,5,4,1,6,5,8,2023,12,8,5
611767,3,14,139,0,7,1,2,3,243,10,3,5,5,1,6,2023,12,4,5
611768,3,14,139,0,7,0,0,2,454,6,2,3,1,7,3,2023,12,13,5


In [11]:
# 라벨 디코딩
for column in label_encoders:
    if column == '사고매개물':
        non_null_mask = data[column].notnull()
        data.loc[non_null_mask, column] = label_encoders[column].inverse_transform(data.loc[non_null_mask, column].astype(int))
    else:
        data[column] = label_encoders[column].inverse_transform(data[column].astype(int))
        data[column] = data[column].replace('NULL_PLACEHOLDER', None)

data

,학교급,지역,교육청,설립유형,사고자구분,사고자성별,사고자학년,사고발생요일,사고발생시각,사고시간,사고장소,사고부위,사고형태,사고당시활동,사고매개물,사고연도,사고월,사고일,나이
0,중학교,경기,안양과천교육지원청,공립,일반학생,남,1학년,수,09:00,수업시간,부속시설,다리,낙상-미끄러짐,기타운동,건물(문/창문/바닥/벽 등),2019,1,2,중학생
1,초등학교,경기,안양과천교육지원청,공립,일반학생,남,2학년,수,08:50,등하교,통로,치아(구강),낙상-넘어짐,보행/주행,건물(문/창문/바닥/벽 등),2019,1,2,초등학생 저학년
2,중학교,경기,부천교육지원청,공립,일반학생,여,1학년,수,10:20,체육수업,부속시설,손,기타,구기운동,기타,2019,1,2,중학생
3,초등학교,광주,서부교육지원청,공립,일반학생,여,2학년,수,10:35,휴식시간 및 청소시간,운동장,머리(두부),낙상-넘어짐,장난/놀이,자연(사람/동물/식물 등),2019,1,2,초등학생 저학년
4,중학교,충북,청주교육지원청,공립,일반학생,남,2학년,수,09:10,체육수업,부속시설,발,낙상-미끄러짐,보행/주행,건물(문/창문/바닥/벽 등),2019,1,2,중학생
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
611765,중학교,제주,서귀포시교육지원청,공립,일반학생,여,2학년,월,14:10,쉬는시간,교실,팔,물리적힘 노출,장난/놀이,가구(책상/의자/책장/탁자/침대 등),2023,12,11,중학생
611766,중학교,제주,제주시교육지원청,공립,일반학생,남,1학년,금,13:20,쉬는시간,통로,다리,사람과의 충돌,식사/수면/휴식,자연(사람/동물/식물 등),2023,12,8,중학생
611767,중학교,제주,제주시교육지원청,공립,일반학생,여,3학년,월,09:20,특별활동,운동장,손,물리적힘 노출,구기운동,운동(놀이)용 장비/기구(공/운동기구/운동장 기구 등),2023,12,4,중학생
611768,중학교,제주,제주시교육지원청,공립,일반학생,남,1학년,수,12:25,식사시간,부속시설,발,낙상,장난/놀이,기타,2023,12,13,중학생


In [12]:
# 결과물을 새로운 파일로 저장
output_file_path = './데이터/predict_사고매개물.xlsx'
data.to_excel(output_file_path, index=False)

# 저장된 파일 경로 출력
print(f'Predicted data saved to {output_file_path}')

Predicted data saved to ./데이터/predict_사고매개물.xlsx
